<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 1. 安裝 gspread 和 pandas 函式庫 (此步驟在 Colab 或 Jupyter 環境中通常需執行)
# !pip install -q gspread pandas requests

import gradio as gr
import pandas as pd
from datetime import datetime
import gspread
# 引入 google 授權相關模組
from google.colab import auth
from google.auth import default
import numpy as np
import requests
import json
import time

# 2. 您的 Google Sheet 資訊
SPREADSHEET_ID = '1_3o11yI2G6013MNVzoIf_3KjLsTCiVXcbmjzoGFWtuA'
WORKSHEET_INDEX = 0

# ⚠️ 3. Google 帳號授權
print("正在進行 Google 授權...")

try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    gc_available = True
except Exception as e:
    print(f"⚠️ 警告：Google 授權或憑證取得失敗，無法連線到 Sheets。錯誤: {e}")
    gc = None
    gc_available = False

# 4. 連線到您的試算表和工作表
worksheet = None
if gc_available:
    try:
        spreadsheet = gc.open_by_key(SPREADSHEET_ID)
        worksheet = spreadsheet.get_worksheet(WORKSHEET_INDEX)
        print("✅ Google Sheets 連線成功！")
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"❌ 錯誤：找不到 ID 為 {SPREADSHEET_ID} 的試算表。")
    except Exception as e:
        print(f"❌ 錯誤：連線或權限不足。詳細錯誤: {e}")

# 預期欄位名稱
GRADE_COLUMNS = ["Date", "Student_ID", "Name", "Subject", "Score"]

# 預先載入資料
df_grades = pd.DataFrame(columns=GRADE_COLUMNS)
if worksheet:
    try:
        data_records = worksheet.get_all_records()
        df_grades = pd.DataFrame(data_records)
        for col in GRADE_COLUMNS:
            if col not in df_grades.columns:
                 df_grades[col] = pd.NA
    except Exception as e:
        df_grades = pd.DataFrame(columns=GRADE_COLUMNS)

# 設置 Dataframe，讓最新紀錄在最上面
initial_df_display = df_grades.iloc[::-1] if not df_grades.empty else df_grades


# --- AI 分析函式 ---

def generate_kpi_analysis(df_analysis: pd.DataFrame) -> str:
    """
    呼叫 Gemini API，根據成績數據生成 AI 績效分析報告。
    """
    # 檢查數據是否充足
    if df_analysis.empty:
        return "⚠️ 數據不足，無法進行 AI 分析。"

    # 1. 準備數據給 LLM
    overall_avg = df_analysis['Score'].mean()

    # 各科目平均
    subject_averages = df_analysis.groupby('Subject')['Score'].mean().sort_values(ascending=False).round(2).to_dict()

    # 學生平均 (用於強弱分析)
    student_averages = df_analysis.groupby('Name')['Score'].mean().round(2).sort_values()

    top_students = student_averages.tail(3).to_dict()
    bottom_students = student_averages.head(3).to_dict()

    # 2. 建構 Prompt
    subject_data_str = json.dumps(subject_averages, ensure_ascii=False)
    top_student_str = json.dumps(top_students, ensure_ascii=False)
    bottom_student_str = json.dumps(bottom_students, ensure_ascii=False)

    user_query = f"""
    請根據以下數據進行成績評估和 KPI 分析，並嚴格按照要求的分段結構輸出結果：

    1. **整體平均分數**: {overall_avg:.2f}
    2. **各科目平均分數**: {subject_data_str}
    3. **平均成績最高的前三名學生**: {top_student_str}
    4. **平均成績最低的後三名學生**: {bottom_student_str}
    """

    system_prompt = (
        "你是一位資深教育績效分析師，專門為學校提供成績評估和學習建議。請根據提供的成績數據，用清晰、結構化的 Markdown 格式，提供以下五個部分的 KPI 分析（使用 ## 標題）。請確保分析結果包含：\n"
        "## 1. 整體成績表現摘要\n"
        "## 2. 各科目的平均成績和分布情況\n"
        "## 3. 學生的強弱科目分析\n"
        "## 4. 學習建議和改進方向\n"
        "## 5. 需要特別關注的科目或學期"
    )

    # 3. API 配置 (使用空字串 API Key, 依規範讓環境自動提供)
    API_KEY = ""
    model_name = "gemini-2.5-flash-preview-05-20"
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent?key={API_KEY}"

    payload = {
        "contents": [{"parts": [{"text": user_query}]}],
        "tools": [{"google_search": {}}],
        "systemInstruction": {"parts": [{"text": system_prompt}]},
    }

    headers = {'Content-Type': 'application/json'}

    # 4. 執行 API 呼叫 (帶有指數退避機制)
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(api_url, headers=headers, data=json.dumps(payload))
            response.raise_for_status()

            result = response.json()

            # 提取生成的文本
            candidate = result.get('candidates', [{}])[0]
            generated_text = candidate.get('content', {}).get('parts', [{}])[0].get('text', 'AI 分析失敗：未從模型獲得有效文本。')

            # 檢查是否有資料來源 (如果使用了 Search grounding)
            sources = []
            grounding_metadata = candidate.get('groundingMetadata')
            if grounding_metadata and grounding_metadata.get('groundingAttributions'):
                sources = grounding_metadata['groundingAttributions']
                # 這裡我們只顯示分析內容，不顯示來源，因為數據是本地提供的。

            return generated_text

        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                time.sleep(wait_time)
            else:
                return f"❌ AI 分析失敗：無法連線到服務或超過重試次數。錯誤詳情: {e}"
        except Exception as e:
            return f"❌ AI 分析失敗：處理回應時發生錯誤。錯誤詳情: {e}"


# --- Gradio 互動函式 ---

def view_grades(filter_text: str = ""):
    """
    讀取成績紀錄，執行篩選，並生成統計總結和 AI 分析報告。
    返回 (統計總結, 數據表格, AI 分析報告)
    """
    if not worksheet:
        return (
            "❌ 錯誤：未成功連線到 Google Sheet，請先執行步驟一。",
            pd.DataFrame(columns=GRADE_COLUMNS),
            "無法執行 AI 分析。"
        )

    try:
        # 1. 重新讀取原始資料
        data_records = worksheet.get_all_records()
        df_full = pd.DataFrame(data_records)

        if df_full.empty or 'Score' not in df_full.columns:
            empty_df = pd.DataFrame(columns=GRADE_COLUMNS)
            return "目前沒有成績紀錄。", empty_df, "數據不足，無法執行 AI 分析。"

        # 2. 資料清理與轉換
        df_full['Score'] = pd.to_numeric(df_full['Score'], errors='coerce')
        df_full.dropna(subset=['Score'], inplace=True)

        if df_full.empty:
            empty_df = pd.DataFrame(columns=GRADE_COLUMNS)
            return "目前沒有有效成績紀錄。", empty_df, "數據不足，無法執行 AI 分析。"

        # 3. 執行篩選
        df_filtered = df_full.copy()
        filter_text = filter_text.strip()

        if filter_text:
            mask = df_filtered.apply(
                lambda row: (
                    filter_text.lower() in str(row['Student_ID']).lower() or
                    filter_text.lower() in str(row['Name']).lower() or
                    filter_text.lower() in str(row['Subject']).lower()
                ), axis=1
            )
            df_filtered = df_filtered[mask]

            if df_filtered.empty:
                empty_df = pd.DataFrame(columns=GRADE_COLUMNS)
                return f"⚠️ 找不到包含 '{filter_text}' 的紀錄。", empty_df, "篩選結果為空，無法執行 AI 分析。"

        # 4. 統計分析 (基於過濾後的資料)
        df = df_filtered
        average_score = df["Score"].mean()
        subject_averages = df.groupby('Subject')['Score'].mean().sort_values(ascending=False).round(2)
        subject_summary = "\n".join([f"  - **{sub}**: {avg:.2f} 分" for sub, avg in subject_averages.items()])
        student_averages = df.groupby('Name')['Score'].mean().sort_values(ascending=False).round(2)
        top_students = student_averages.head(5)
        student_summary = "\n".join([f"  - **{name}**: {avg:.2f} 分" for name, avg in top_students.items()])

        # 組合結構化總結文字
        summary_text = (
            f"## 📊 成績概況 ({'篩選結果' if filter_text else '所有紀錄'})\n\n"
            f"**總紀錄數：** {len(df)} 筆\n"
            f"**整體平均分數：** **{average_score:.2f} 分**\n\n"
            f"## 📚 各科目平均分數\n"
            f"{subject_summary}\n\n"
            f"## 🏆 成績前 5 名學生 (平均)\n"
            f"{student_summary}"
        )

        # 5. 執行 AI KPI 分析
        # ⚠️ 注意：這裡將 df_full 傳遞給 AI 進行整體分析，即使有篩選，
        # 也可以給予 AI 完整的數據上下文。
        kpi_analysis_text = generate_kpi_analysis(df_full)

        # 6. 重新命名欄位以符合中文顯示
        df_display = df.rename(columns={
            "Date": "日期 (年-月-日 時:分:秒)",
            "Student_ID": "學號",
            "Name": "姓名",
            "Subject": "科目",
            "Score": "成績"
        })

        # 返回三個結果
        return summary_text, df_display.iloc[::-1], kpi_analysis_text

    except Exception as e:
        return f"❌ 讀取或分析 Google Sheets 失敗: {e}", pd.DataFrame(columns=GRADE_COLUMNS), "無法執行 AI 分析。"


def record_grade(student_id: str, name: str, subject: str, score: float):
    """
    紀錄一筆新的成績，並儲存到 Google Sheet 中。
    返回 (狀態訊息, 數據表格, AI 分析報告)
    """
    if not worksheet:
        _, current_df, _ = view_grades() # 取得當前 DataFrame 以保持介面穩定
        return "❌ 錯誤：未成功連線到 Google Sheet。", current_df, "無法執行 AI 分析。"

    if not (0 <= score <= 100):
        _, current_df, current_kpi = view_grades()
        return "❌ 錯誤：成績必須介於 0 到 100 之間。", current_df, current_kpi

    if not student_id or not name:
        _, current_df, current_kpi = view_grades()
        return "❌ 錯誤：學號和姓名不能為空。", current_df, current_kpi

    date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_row = [date_time, student_id, name, subject, score]

    try:
        worksheet.append_row(new_row, value_input_option='USER_ENTERED')

        # 刷新並取得最新的資料和分析
        summary_text, updated_df, kpi_analysis_text = view_grades()
        return f"✅ 成功紀錄: 學號 {student_id}, {name} 的 {subject} 成績: {score} 分", updated_df, kpi_analysis_text

    except Exception as e:
        _, current_df, current_kpi = view_grades()
        return f"❌ 儲存到 Google Sheets 失敗: {e}", current_df, current_kpi


# --- Gradio 介面設定 ---

subjects = [
    "國文",
    "英文",
    "數學",
    "科學",
    "社會",
    "電腦"
]

# 由於 view_grades 現在返回三個值，我們需要一個初始值來佔位 AI 分析的輸出。
initial_summary, initial_df_display, initial_kpi = view_grades()

with gr.Blocks(title="Google Sheets 成績紀錄系統") as demo:
    gr.Markdown("# 📝 Google Sheets 學生分數紀錄系統 (含 AI KPI 分析)")

    # 紀錄成績 Tab
    with gr.Tab("紀錄成績"):
        gr.Markdown("### 填寫學生與成績資訊 (自動紀錄日期與時間)")
        with gr.Row():
            student_id_input = gr.Textbox(label="學號", placeholder="例如：S123456", scale=1)
            name_input = gr.Textbox(label="姓名", placeholder="例如：王小明", scale=1)

        with gr.Row():
            subject_input = gr.Dropdown(subjects, label="科目", value=subjects[0], scale=1)
            score_input = gr.Number(label="成績 (0-100)", minimum=0, maximum=100, scale=1)

        record_button = gr.Button("新增成績紀錄", variant="primary")
        status_output = gr.Label(label="狀態/提示訊息")

    # 查看成績 Tab
    with gr.Tab("查看所有成績"):
        gr.Markdown("### 🔍 成績查詢與分析")

        # 篩選輸入框
        filter_input = gr.Textbox(
            label="篩選紀錄",
            placeholder="輸入學號、姓名或科目關鍵字進行查詢 (例如：小明 或 數學)"
        )

        view_button = gr.Button("執行查詢 / 重新整理並分析紀錄", variant="secondary")

        summary_output = gr.Markdown(value=initial_summary, label="統計總結分析")

        # 新增 AI 分析輸出欄位
        kpi_analysis_output = gr.Markdown(value=initial_kpi, label="🤖 AI 績效分析報告 (KPI)")

        dataframe_output = gr.DataFrame(
            value=initial_df_display,
            headers=["日期 (年-月-日 時:分:秒)", "學號", "姓名", "科目", "成績"],
            wrap=True,
            interactive=False,
            label="所有成績紀錄 (最新在最上)"
        )

    # --- 互動邏輯 ---
    # 紀錄按鈕現在要更新三個輸出
    record_button.click(
        fn=record_grade,
        inputs=[student_id_input, name_input, subject_input, score_input],
        outputs=[status_output, dataframe_output, kpi_analysis_output]
    )

    # 查看按鈕現在要更新三個輸出
    view_button.click(
        fn=view_grades,
        inputs=[filter_input],
        outputs=[summary_output, dataframe_output, kpi_analysis_output]
    )

    # 介面初始載入時也更新三個輸出
    demo.load(
        fn=view_grades,
        inputs=[gr.State("")],
        outputs=[summary_output, dataframe_output, kpi_analysis_output]
    )

# 啟動 Gradio 介面
demo.launch(share=True)

正在進行 Google 授權...
✅ Google Sheets 連線成功！
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a63311935d092d8e46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
